In [25]:
import os
import warnings
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
warnings.filterwarnings('ignore')

os.environ["TAVILY_API_KEY"]=TAVILY_API_KEY
os.environ['OPENAI_API_KEY']=OPENAI_API_KEY

In [27]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

class State(TypedDict):
    messages: Annotated[list, add_messages]

tool = TavilySearchResults(max_results=2)
tools=[tool]
tool_node = ToolNode(tools)

llm = ChatOpenAI(model = "gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    result = llm_with_tools.invoke(state["messages"])
    return {"messages": [result]}

graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)

graph_builder.set_entry_point("chatbot")
graph=graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"]
)

In [30]:
user_input = "Langgraph가 뭐야?"
config={"configurable": {"thread_id": "2"}}

events = graph.stream(
    {
        "messages": [("user", user_input)]
    },
    config,
    stream_mode="values"
)

In [31]:
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Langgraph가 뭐야?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_P0kp3jdyDGaqGqzZPVUI6vC1)
 Call ID: call_P0kp3jdyDGaqGqzZPVUI6vC1
  Args:
    query: Langgraph
